#Importações e inicialização do Pygame


In [1]:
import pygame
import random
import time

# Inicialização do Pygame
pygame.init()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


(5, 0)

#Variáveis

In [2]:
simulation_speed = 0.01  # Velocidade inicial de 1 segundo
initial_rabbit_count = 10  # Quantidade inicial de coelhos
initial_plant_count = 100  # Quantidade inicial de plantas
plant_generation_rate = 10  # Quantidade de plantas geradas por epoca

SCREEN_WIDTH, SCREEN_HEIGHT = 1920, 1200 # Definir o tamanho da tela

# Função para converter cor hexadecimal para RGB
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')  # Remove o "#" se estiver presente
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

# Definir as cores
WHITE = hex_to_rgb("#838383")
BLACK = hex_to_rgb("#000000")
GREEN = hex_to_rgb("#00FF00")
ORANGE = hex_to_rgb("#FFA500")
BABY_RABBIT_COLOR = hex_to_rgb("#0000FF")

# Definir o tamanho da grid e o tamanho da célula
GRID_SIZE_METERS = 40  # 40 metros (lado)
CELL_SIZE = 20  # Cada célula será 20x20 pixels
GRID_SIZE = GRID_SIZE_METERS * CELL_SIZE  # Tamanho total da grid em pixels

# Calcular a posição para centralizar a grid
GRID_X = (SCREEN_WIDTH - GRID_SIZE) // 2
GRID_Y = (SCREEN_HEIGHT - GRID_SIZE) // 2

# Criar a janela
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Grid Centralizada")

#Class Plant


In [3]:
# Classe Plant
class Plant:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def generate_plants(num_plants, occupied_positions):
        plants = []
        available_positions = [(x, y) for x in range(GRID_SIZE_METERS) for y in range(GRID_SIZE_METERS) if (x, y) not in occupied_positions]

        if len(available_positions) == 0:
            print("Nenhum espaço disponível para novas plantas!")  # Apenas exibe se não há espaço livre
            return plants  # Nenhuma planta pode ser gerada agora

        num_plants = min(num_plants, len(available_positions))  # Limitar o número de plantas ao espaço disponível

        for _ in range(num_plants):
            x, y = random.choice(available_positions)
            plants.append(Plant(x, y))
            occupied_positions.add((x, y))  # Marcar a posição como ocupada
            available_positions.remove((x, y))  # Remover a posição da lista de disponíveis

        return plants


#Classe Rabbit

In [4]:
# Classe rabbit
class Rabbit:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.lifespan = 20  # Vida do coelho em ciclos (20 ciclos)
        self.age = 0  # Idade do coelho
        self.is_baby = True  # Indica se o coelho é recém-nascido
    
    def move(self):
        direction = random.choice(['up', 'down', 'left', 'right'])
        if direction == 'up' and self.y > 0:
            self.y -= 1
        elif direction == 'down' and self.y < GRID_SIZE_METERS - 1:
            self.y += 1
        elif direction == 'left' and self.x > 0:
            self.x -= 1
        elif direction == 'right' and self.x < GRID_SIZE_METERS - 1:
            self.x += 1

    def eat_plants_and_reproduce(self, plants, rabbits, occupied_positions):
        plants_to_remove = []
        for plant in plants:
            if (plant.x, plant.y) == (self.x, self.y):  # Checagem mais simples
                plants_to_remove.append(plant)
                occupied_positions.discard((plant.x, plant.y))  # Remover planta ocupada
                self.reproduce(rabbits, occupied_positions)  # Reproduzir após comer planta
                print(f"Coelho em ({self.x}, {self.y}) comeu planta em ({plant.x}, {plant.y})")

        
        # Remover as plantas comidas
        for plant in plants_to_remove:
            plants.remove(plant)



    def reproduce(self, rabbits, occupied_positions):
        directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        available_positions = [(self.x + dx, self.y + dy) for dx, dy in directions
                            if 0 <= self.x + dx < GRID_SIZE_METERS and 0 <= self.y + dy < GRID_SIZE_METERS]

        # Certifique-se de verificar todas as direções, mesmo que estejam ocupadas inicialmente
        available_positions = [pos for pos in available_positions if pos not in occupied_positions]

        # Aumente o número de direções caso precise de mais flexibilidade
        if available_positions:
            new_x, new_y = random.choice(available_positions)
            new_rabbit = Rabbit(new_x, new_y)
            rabbits.append(new_rabbit)
            occupied_positions.add((new_x, new_y))  # Marcar nova posição ocupada

    
    def age_rabbit(self):
        self.age += 1
        return self.age < self.lifespan


#Funções de Desenho

In [5]:
# Funções de desenho
def generate_plants(num_plants, occupied_positions):
    plants = []
    available_positions = [(x, y) for x in range(GRID_SIZE_METERS) for y in range(GRID_SIZE_METERS) if (x, y) not in occupied_positions]

    if len(available_positions) < num_plants:
        num_plants = len(available_positions)  # Limitar o número de plantas se o espaço disponível for menor

    for _ in range(num_plants):
        x, y = random.choice(available_positions)
        plants.append(Plant(x, y))
        occupied_positions.add((x, y))  # Marcar a posição como ocupada
        available_positions.remove((x, y))  # Remover a posição da lista de disponíveis

    return plants

def draw_grid():
    for row in range(GRID_SIZE_METERS):
        for col in range(GRID_SIZE_METERS):
            rect = pygame.Rect(GRID_X + col * CELL_SIZE, GRID_Y + row * CELL_SIZE, CELL_SIZE, CELL_SIZE)
            pygame.draw.rect(screen, BLACK, rect, 1)

def draw_plants(plants):
    for plant in plants:
        rect = pygame.Rect(GRID_X + plant.x * CELL_SIZE, GRID_Y + plant.y * CELL_SIZE, CELL_SIZE, CELL_SIZE)
        pygame.draw.rect(screen, GREEN, rect)

def draw_rabbits(rabbits):
    for rabbit in rabbits:
        color = BABY_RABBIT_COLOR if rabbit.is_baby else ORANGE
        rect = pygame.Rect(GRID_X + rabbit.x * CELL_SIZE, GRID_Y + rabbit.y * CELL_SIZE, CELL_SIZE, CELL_SIZE)
        pygame.draw.rect(screen, color, rect)

def draw_legend(plant_count, rabbit_count, x, y):
    font = pygame.font.SysFont(None, 48)  # Fonte padrão
    plant_count_text = font.render(f"Plantas: {plant_count}", True, BLACK)
    rabbit_count_text = font.render(f"Coelhos: {rabbit_count}", True, BLACK)

    plant_text_rect = plant_count_text.get_rect(topleft=(x, y))
    screen.blit(plant_count_text, plant_text_rect)  # Desenhar a legenda das plantas
    rabbit_text_rect = rabbit_count_text.get_rect(topleft=(x, y + 50))
    screen.blit(rabbit_count_text, rabbit_text_rect)  # Desenhar a legenda dos coelhos

def draw_button(text, x, y, width, height, active):
    color = BLACK if active else WHITE
    pygame.draw.rect(screen, color, (x, y, width, height), 0)
    pygame.draw.rect(screen, BLACK, (x, y, width, height), 3)
    font = pygame.font.SysFont(None, 48)
    text_surface = font.render(text, True, BLACK)
    screen.blit(text_surface, (x + (width - text_surface.get_width()) // 2, y + (height - text_surface.get_height()) // 2))

# Inicializa a matriz de ocupação 40x40 com zeros
grid_occupancy = [[0 for _ in range(GRID_SIZE_METERS)] for _ in range(GRID_SIZE_METERS)]

def update_grid_occupancy(plants, rabbits):
    # Zera a matriz de ocupação
    for i in range(GRID_SIZE_METERS):
        for j in range(GRID_SIZE_METERS):
            grid_occupancy[i][j] = 0

    # Marca plantas na matriz
    for plant in plants:
        grid_occupancy[plant.x][plant.y] = 1

    # Marca coelhos na matriz
    for rabbit in rabbits:
        grid_occupancy[rabbit.x][rabbit.y] = 2

def print_grid_occupancy():
    # Imprime a matriz de ocupação para debug
    print("Matriz de Ocupação:")
    for row in grid_occupancy:
        print(" ".join(map(str, row)))
        print("\n")

#Função Principal

In [6]:
# Função Principal
def main():
    clock = pygame.time.Clock()
    running = True
    show_grid = False
    plants = []  # Lista de plantas
    rabbits = []
    
    occupied_positions = set()  # Conjunto de posições ocupadas

    # Gerar coelhos inicialmente
    for _ in range(initial_rabbit_count):
        while True:
            x, y = random.randint(0, GRID_SIZE_METERS-1), random.randint(0, GRID_SIZE_METERS-1)
            if (x, y) not in occupied_positions:
                rabbits.append(Rabbit(x, y))
                occupied_positions.add((x, y))
                break
    
    last_plant_time = time.time()
    last_rabbit_move = time.time()
    simulation_start_time = time.time()  # Armazena o tempo em que a simulação começou
    epochs = 0  # Contador de épocas

    # Variável de controle para impedir que o botão seja clicável após o início da simulação
    simulation_started = False

    # Botão "Start" centralizado
    button_rect = pygame.Rect(SCREEN_WIDTH // 2 - 100, SCREEN_HEIGHT // 2 - 50, 200, 100)

    while running:
        screen.fill(WHITE)  # Cor de fundo

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            if event.type == pygame.MOUSEBUTTONDOWN:
                # Permitir o clique no botão "Start" apenas se a simulação ainda não tiver começado
                if not simulation_started and button_rect.collidepoint(event.pos):
                    show_grid = True  # Após o clique, mostra o grid
                    plants = generate_plants(initial_plant_count, occupied_positions)  # Gerar plantas inicialmente
                    rabbits = [Rabbit(random.randint(0, GRID_SIZE_METERS-1), random.randint(0, GRID_SIZE_METERS-1)) for _ in range(initial_rabbit_count)]  # Gerar coelhos inicialmente
                    simulation_started = True  # Marca que a simulação começou

        if show_grid:
            draw_grid()  # Desenhar grid
            
            # Gerar novas plantas baseado na velocidade
            if time.time() - last_plant_time >= simulation_speed:  
                new_plants = generate_plants(plant_generation_rate, occupied_positions)
                plants.extend(new_plants)
                last_plant_time = time.time()

            # Mover os coelhos e fazer com que eles comam plantas
            if time.time() - last_rabbit_move >= simulation_speed:  
                for rabbit in rabbits:
                    rabbit.move()
                    # Aqui, os coelhos comerão plantas e poderão se reproduzir
                    rabbit.eat_plants_and_reproduce(plants, rabbits, occupied_positions) 
                    rabbit.is_baby = rabbit.age < 1  # Definir se é recém-nascido
                    if not rabbit.age_rabbit():
                        rabbits.remove(rabbit)
                last_rabbit_move = time.time()
                epochs += 1  # Incrementar o contador de épocas após cada movimento

            draw_plants(plants)  # Desenhar plantas
            draw_rabbits(rabbits)  # Desenhar coelhos
            draw_legend(len(plants), len(rabbits), 220, 150)  # Exibe a legenda com a quantidade de plantas e coelhos

            # Calcular o tempo de simulação
            simulation_time = int(time.time() - simulation_start_time)
            font = pygame.font.SysFont(None, 36)
            time_text = font.render(f"Tempo de simulação: {simulation_time} segundos", True, BLACK)
            epoch_text = font.render(f"Épocas realizadas: {epochs}", True, BLACK)
            screen.blit(time_text, (220, 220))  # Exibir o tempo de simulação
            screen.blit(epoch_text, (220, 250))  # Exibir o número de épocas

        else:
            draw_button("Start", button_rect.x, button_rect.y, button_rect.width, button_rect.height, False)

        pygame.display.flip()
        clock.tick(60)  # Limitar a 60 FPS

    pygame.quit()

# Rodar o jogo
main()


Coelho em (26, 19) comeu planta em (26, 19)
Coelho em (22, 9) comeu planta em (22, 9)
Coelho em (21, 7) comeu planta em (21, 7)
Coelho em (33, 36) comeu planta em (33, 36)
Coelho em (37, 2) comeu planta em (37, 2)
Coelho em (14, 14) comeu planta em (14, 14)
Coelho em (25, 6) comeu planta em (25, 6)
Coelho em (34, 2) comeu planta em (34, 2)
Coelho em (20, 3) comeu planta em (20, 3)
Coelho em (24, 20) comeu planta em (24, 20)
Coelho em (28, 18) comeu planta em (28, 18)
Coelho em (37, 33) comeu planta em (37, 33)
Coelho em (25, 8) comeu planta em (25, 8)
Coelho em (37, 32) comeu planta em (37, 32)
Coelho em (26, 9) comeu planta em (26, 9)
Coelho em (27, 10) comeu planta em (27, 10)
Coelho em (13, 39) comeu planta em (13, 39)
Coelho em (35, 39) comeu planta em (35, 39)
Coelho em (27, 9) comeu planta em (27, 9)
Coelho em (28, 10) comeu planta em (28, 10)
Coelho em (14, 32) comeu planta em (14, 32)
Coelho em (13, 38) comeu planta em (13, 38)
Coelho em (31, 5) comeu planta em (31, 5)
Coelho e